## 测试base_rkb_pretask中的各类函数效果

In [1]:
import numpy as np
import torch
import torchio.transforms
from tqdm import tqdm
import os
import glob
import SimpleITK as sitk
from scipy import ndimage


ModuleNotFoundError: No module named 'torchio'

### base_rkb_pretask定义的各类函数

In [ ]:
def crop_3d(self, image, flag, crop_size): # ...（对图像执行3D裁剪的方法）
    h, w, d = crop_size[0], crop_size[1], crop_size[2]
    h_old, w_old, d_old = image.shape[0], image.shape[1], image.shape[2]

    if flag == 'train': #如果 flag 是 'train'，则进行随机裁剪。在这种情况下，生成随机的裁剪起始点 (x, y, z)，h_old-h是为了确保裁剪框不超出图像边界。
        # crop random
        x = np.random.randint(0, 1 + h_old - h) #返回一个随机数或随机数数组(指定size时)
        y = np.random.randint(0, 1 + w_old - w)
        z = np.random.randint(0, 1 + d_old - d)
    else: #如果 flag 不是 'train'，则进行中心裁剪。计算裁剪起始点 (x, y, z) 使得裁剪的区域在图像中居中。
        # crop center
        x = int((h_old - h) / 2)
        y = int((w_old - w) / 2)
        z = int((d_old - d) / 2)

    return self.do_crop_3d(image, x, y, z, h, w, d)

def do_crop_3d(self, image, x, y, z, h, w, d): #3D裁剪的辅助函数，确保整数和返回对应位置的3D块
    assert type(x) == int, x
    assert type(y) == int, y
    assert type(z) == int, z
    assert type(h) == int, h
    assert type(w) == int, w
    assert type(d) == int, d

    return image[x:x + h, y:y + w, z:z + d] #这边取的时最边界点加上crop_size的大小，解释了为何上述代码的定中心方式如此

def crop_cubes_3d(self, image, flag, cubes_per_side, cube_jitter_xy=3, cube_jitter_z=3): #在3D图像中裁剪多个3D立方体的方法，jitter貌似是为了防止数据连续变化留出的间隔大小，cubes_per_side应该是每个方向有多少个cube
    h, w, d = image.shape

    patch_overlap = -cube_jitter_xy if cube_jitter_xy < 0 else 0 #patch_overlap 计算了在裁剪时可能存在的重叠区域。如果 cube_jitter_xy 是负数，则 patch_overlap 采用该值；否则，为零。

    #h_grid、w_grid 和 d_grid 计算了每个立方体的网格大小，以确保裁剪时没有重叠区域。
    # 这里主要思想应该是把切出来的3D大块划分为各个小块,这里拿笔记做做数学题吧，好久没写了。
    h_grid = (h - patch_overlap) // cubes_per_side
    w_grid = (w - patch_overlap) // cubes_per_side
    d_grid = (d - patch_overlap) // cubes_per_side
    h_patch = h_grid - cube_jitter_xy
    w_patch = w_grid - cube_jitter_xy
    d_patch = d_grid - cube_jitter_z

    cubes = []
    for i in range(cubes_per_side):
        for j in range(cubes_per_side):
            for k in range(cubes_per_side):

                p = self.do_crop_3d(image, #当i=0时，从0截取到第一个grid
                                i * h_grid,
                                j * w_grid,
                                k * d_grid,
                                h_grid + patch_overlap,
                                w_grid + patch_overlap,
                                d_grid + patch_overlap)

                if h_patch < h_grid or w_patch < w_grid or d_patch < d_grid:
                    p = self.crop_3d(p, flag, [h_patch, w_patch, d_patch])

                cubes.append(p)

    return cubes

def rearrange(self, cubes, K_permutations):  # ...（根据排列重新排列立方体的方法）
    label = random.randint(0, len(K_permutations) - 1)
    # print('label', np.array(K_permutations[label]), label)
    return np.array(cubes)[np.array(K_permutations[label])], label

def center_crop_xy(self, image, size): # 在image中间截一个size大小的缺口，画图很容易理解
    """CenterCrop a sample.
        Args:
            image: [D, H, W]
            label:[D, H, W]
            crop_size: the desired output size in the x-y plane
        Returns:
            out_image:[D, h, w]
            out_label:[D, h, w]
    """
    h, w, d = image.shape

    h1 = int(round((h - size[0]) / 2.)) #round() 函数将结果四舍五入为最接近的整数，确保裁剪区域的起始点是整数。
    w1 = int(round((w - size[1]) / 2.))

    image = image[h1:h1 + size[0], w1:w1 + size[1], :]
    return image

def rotate(self, cubes): # ...（旋转3D立方体的方法）

    # multi-hot labels
    # [8, H, W, D]
    rot_cubes = copy.deepcopy(cubes)
    hor_vector = []
    ver_vector = []

    for i in range(self.num_cubes):
        p = random.random()
        cube = rot_cubes[i]
        # [H, W, D]
        if p < 1/3:
            hor_vector.append(1)
            ver_vector.append(0)
            # rotate 180 along x axis
            rot_cubes[i] = np.flip(cube, (1, 2))
        elif p < 2/3:
            hor_vector.append(0)
            ver_vector.append(1)
            # rotate 180 along z axis
            rot_cubes[i] = np.flip(cube, (0, 1))

        else:
            hor_vector.append(0)
            ver_vector.append(0)

    return rot_cubes, hor_vector, ver_vector

def mask(self, cubes): # ...（对3D立方体应用掩码的方法）
    mask_vector = []
    masked_cubes = copy.deepcopy(cubes)
    for i in range(self.num_cubes):
        cube = masked_cubes[i]
        if random.random() < 0.5:
            # mask
            mask_vector.append(1)
            R = np.random.uniform(0, 1, cube.shape)
            R = (R > 0.5).astype(np.int32)
            masked_cubes[i] = cube * R
        else:
            mask_vector.append(0)

    return masked_cubes, mask_vector

In [ ]:


input = torch.rand(240,240,155)
print(input.shape)


In [ ]:
input = np.load(img_path)
## input:  [320, 320, 74]

if self.crop_size == [128, 128, 32]: #如果刚好是中心裁剪的尺寸，刚好裁成4块
# input: [276, 276, 74]
    input = self.center_crop_xy(input, [276, 276]) #先对xy平面截取中心缺口从320，320，74截取到276，276，74

    # get all the num_grids **3 cubes
    all_cubes = self.crop_cubes_3d(input,
                                flag=self.flag,
                                cubes_per_side=self.num_grids_per_axis,
                                cube_jitter_xy=10,
                                cube_jitter_z=5)
# print(len(all_cubes), all_cubes[0].shape)

elif self.crop_size == [64, 64, 16]:
    # input: [140, 140, 40]
    input = ndimage.zoom(input, [140/320, 140/320, 40/74], order=3)

    # get all the num_grids **3 cubes
    all_cubes = self.crop_cubes_3d(input,
                                           flag=self.flag,
                                           cubes_per_side=self.num_grids_per_axis,
                                           cube_jitter_xy=6,
                                           cube_jitter_z=4)

    else:
    print('This crop size has not been configured yet')
            all_cubes = None

        # Task1: Permutate the order of cubes
        rearranged_cubes, order_label = self.rearrange(all_cubes, self.K_permutations)

        # Task2: Rotate each cube randomly.
        rearranged_cubes, hor_label, ver_label = self.rotate(rearranged_cubes)

        final_cubes = np.expand_dims(np.array(rearranged_cubes), axis=1)

        return torch.from_numpy(final_cubes.astype(np.float32)), \
               torch.from_numpy(np.array(order_label)),\
               torch.from_numpy(np.array(hor_label)).float(), \
               torch.from_numpy(np.array(ver_label)).float()

    def get_luna_list(self):
        self.all_images = []
        for index_subset in self.folds:
            luna_subset_path = os.path.join(self.base_dir, "subset" + str(index_subset))
            file_list = glob.glob(os.path.join(luna_subset_path, "*.npy"))
            # save the path
            for img_file in tqdm(file_list):
                self.all_images.append(img_file)
        # x_train: (445)
        # x_valid: (178)
        return
